In [167]:
import os
import sys
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [168]:
results_dir = './results'

In [169]:
results_files = list()
overall_results_df = None
results_files_columns = ['method', 'model', 'system', 'prompt', 'temperature', 'run', 'dataset']

for results_file in sorted(os.listdir(results_dir)):
    if results_file.endswith('overall.csv') or results_file.endswith('granular.csv'):
        continue
    
    results_file_parts = results_file.split('.csv')[0].split('_')
    
    system, prompt, temperature, run = ['NA']*4

    if results_file_parts[0] == 'code-llama-34b':
        method = 'LLM'
        model = results_file_parts[0]
        system, prompt, temperature = results_file_parts[1:4]
        dataset = results_file_parts[4]
        run = results_file_parts[5]
    elif results_file_parts[0] == 'OpenAI':
        method = 'LLM'
        model = results_file_parts[1]
        system, prompt, temperature = results_file_parts[2:5]
        dataset = results_file_parts[5]
        run = results_file_parts[6]
    elif results_file_parts[0] == 'exact':
        method = results_file_parts[0]
        model = 'exact'
        dataset = results_file_parts[1]
    elif results_file_parts[0] == 'deepcadrme':
        method = 'BERT'
        model = 'deepcadrme'
        system = results_file_parts[1]
        dataset = results_file_parts[2]
    else:
        raise Exception(results_file_parts[0])

    results_files.append([method, model, system, prompt, temperature, run, dataset])

    for evaltype in ('strict', 'lenient'):
        overall_results_file = results_file.replace('.csv', f"_{evaltype}_overall.csv")
        res = pd.read_csv(os.path.join(results_dir, overall_results_file), index_col=0)
        for name, colval in zip(results_files_columns, results_files[-1]):
            res.insert(0, name, [colval]*res.shape[0])
        res.insert(0, 'evaltype', [evaltype]*res.shape[0])
        
        if overall_results_df is None:
            overall_results_df = res
        else:
            overall_results_df = pd.concat([overall_results_df, res])

results_files_df = pd.DataFrame(results_files, columns=results_files_columns)

In [170]:
results_files_df.groupby(['dataset', 'method', 'model', 'system', 'prompt', 'temperature']).count()

run
dataset method model              system          prompt           temperature     
test    BERT   deepcadrme         005             NA               NA             1
                                  010             NA               NA             1
                                  015             NA               NA             1
                                  025             NA               NA             1
                                  050             NA               NA             1
                                  075             NA               NA             1
                                  100             NA               NA             1
        LLM    gpt-3.5-turbo-0125 fatal-prompt-v2 pharmexpert-v1   temp0          3
               gpt-4-1106-preview fatal-prompt-v2 pharmexpert-v1   temp0          2
                                  fatal-prompt-v3 pharmexpert-v1   temp0          1
        exact  exact              NA              NA               NA             1
train   LLM    code-llama-34b     fatal-prompt-v2 pharmexpert-v0   temp0          1
                                                  pharmexpert-v1   temp0          1
               gpt-3.5-turbo-0125 fatal-prompt-v2 no-system-prompt temp0          3
                                                  pharmexpert-v0   temp0          3
                                                  pharmexpert-v1   temp0          4
               gpt-4-1106-preview fatal-prompt-v2 no-system-prompt temp0          3
                                                  pharmexpert-v0   temp0          3
                                                  pharmexpert-v1   temp0          5
                                  fatal-prompt-v3 pharmexpert-v1   temp0          1
        exact  exact              NA              NA               NA             1

In [246]:
# default eval
groupbycols = ['dataset', 'ade_type', 'section', 'method', 'model', 'system', 'prompt']
additional_querystr = " & ade_type=='all'"

# evaluate the prompt
# found that pharmexpert-v1 is working a bit better than the others
# groupbycols = ['dataset', 'ade_type', 'section', 'prompt', 'method']
# additional_querystr = " & model != 'code-llama-34b' & method == 'LLM'"

# compare ade_types
# groupbycols = ['ade_type', 'dataset', 'ade_type', 'section', 'method', 'model', 'system', 'prompt']
# additional_querystr = ""

querystr = "evaltype=='lenient' & dataset=='test' & section == 'adverse reactions'" + additional_querystr
metric = 'recall'
overall_results_df.query(querystr).groupby(groupbycols)[f"macro_{metric}"].agg(['count', 'mean', 'min', 'max'])

count  \
dataset ade_type section           method model              system          prompt                  
test    all      adverse reactions BERT   deepcadrme         005             NA                  1   
                                                             010             NA                  1   
                                                             015             NA                  1   
                                                             025             NA                  1   
                                                             050             NA                  1   
                                                             075             NA                  1   
                                                             100             NA                  1   
                                   LLM    gpt-3.5-turbo-0125 fatal-prompt-v2 pharmexpert-v1      3   
                                          gpt-4-1106-preview fatal-prompt-v2 pharmexpert-v1      2   
                                                             fatal-prompt-v3 pharmexpert-v1      1   
                                   exact  exact              NA              NA                  1   

                                                                                                 mean  \
dataset ade_type section           method model              system          prompt                     
test    all      adverse reactions BERT   deepcadrme         005             NA              0.962558   
                                                             010             NA              0.971020   
                                                             015             NA              0.962563   
                                                             025             NA              0.962357   
                                                             050             NA              0.963310   
                                                             075             NA              0.965114   
                                                             100             NA              0.965640   
                                   LLM    gpt-3.5-turbo-0125 fatal-prompt-v2 pharmexpert-v1  0.763918   
                                          gpt-4-1106-preview fatal-prompt-v2 pharmexpert-v1  0.850810   
                                                             fatal-prompt-v3 pharmexpert-v1  0.854655   
                                   exact  exact              NA              NA              0.922800   

                                                                                                  min  \
dataset ade_type section           method model              system          prompt                     
test    all      adverse reactions BERT   deepcadrme         005             NA              0.962558   
                                                             010             NA              0.971020   
                                                             015             NA              0.962563   
                                                             025             NA              0.962357   
                                                             050             NA              0.963310   
                                                             075             NA              0.965114   
                                                             100             NA              0.965640   
                                   LLM    gpt-3.5-turbo-0125 fatal-prompt-v2 pharmexpert-v1  0.754800   
                                          gpt-4-1106-preview fatal-prompt-v2 pharmexpert-v1  0.848175   
                                                             fatal-prompt-v3 pharmexpert-v1  0.854655   
                                   exact  exact              NA              NA              0.922800   

                                  

In [247]:
overall_results_df.query(querystr).groupby(groupbycols)[f"micro_{metric}"].agg(['count', 'mean', 'min', 'max'])

count  \
dataset ade_type section           method model              system          prompt                  
test    all      adverse reactions BERT   deepcadrme         005             NA                  1   
                                                             010             NA                  1   
                                                             015             NA                  1   
                                                             025             NA                  1   
                                                             050             NA                  1   
                                                             075             NA                  1   
                                                             100             NA                  1   
                                   LLM    gpt-3.5-turbo-0125 fatal-prompt-v2 pharmexpert-v1      3   
                                          gpt-4-1106-preview fatal-prompt-v2 pharmexpert-v1      2   
                                                             fatal-prompt-v3 pharmexpert-v1      1   
                                   exact  exact              NA              NA                  1   

                                                                                                 mean  \
dataset ade_type section           method model              system          prompt                     
test    all      adverse reactions BERT   deepcadrme         005             NA              0.960137   
                                                             010             NA              0.966781   
                                                             015             NA              0.960994   
                                                             025             NA              0.961852   
                                                             050             NA              0.960780   
                                                             075             NA              0.962495   
                                                             100             NA              0.962709   
                                   LLM    gpt-3.5-turbo-0125 fatal-prompt-v2 pharmexpert-v1  0.739534   
                                          gpt-4-1106-preview fatal-prompt-v2 pharmexpert-v1  0.841299   
                                                             fatal-prompt-v3 pharmexpert-v1  0.843978   
                                   exact  exact              NA              NA              0.921775   

                                                                                                  min  \
dataset ade_type section           method model              system          prompt                     
test    all      adverse reactions BERT   deepcadrme         005             NA              0.960137   
                                                             010             NA              0.966781   
                                                             015             NA              0.960994   
                                                             025             NA              0.961852   
                                                             050             NA              0.960780   
                                                             075             NA              0.962495   
                                                             100             NA              0.962709   
                                   LLM    gpt-3.5-turbo-0125 fatal-prompt-v2 pharmexpert-v1  0.736820   
                                          gpt-4-1106-preview fatal-prompt-v2 pharmexpert-v1  0.837334   
                                                             fatal-prompt-v3 pharmexpert-v1  0.843978   
                                   exact  exact              NA              NA              0.921775   

                                  